# Prediction with LSTM model (v4.5.8)
<p style='text-align: right;'>with selectd5.csv</p>

* history
  * 2025/05/23 PM06:02 : 3번째 모델의 3번째 데이터를 사용
  * 2025/05/27
  * 2025/05/30 : v4.5.1 작성 시작
  * 2025/06/04 : v4.5.2 last update
  * 2025/06/04 : v4.5.3 작성
  * 2025/06/05 : v4.5.4 작성
  * 2025/06/05 : v4.5.5 작성 model_id=14
  * 2025/06/05 : v4.5.6 작성 model_id=4
  * 2025/06/05 : v4.5.7 작성 model_id=2
  * 2025/06/05 : v4.5.7 작성 model_id=8

In [ ]:
!pip install scikit-learn
!pip install tensorflow-cpu
!pip install numpy pandas keras scikit-learn matplotlib scikeras
from datetime import datetime
print(f'restart kernel... {datetime.now()}')

In [3]:
def restart_kernel():
    # Restart the kernet after libraries are loaded.
    import IPython
    from datetime import datetime
    print(f'restart kernel... {datetime.now()}')
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

restart_kernel()

import os
import sys
from datetime import datetime
cwd = os.getcwd()
sys.path.append(cwd)
print(sys.path)

restart kernel... 2025-06-05 23:30:49.740578
['/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/simpson/.local/lib/python3.8/site-packages', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages', '/home/simpson/jupyter-workspace/LSTM_Analyze']


In [1]:
# load dependacies
import os
import time
import resource
from datetime import datetime
from lib.globalvar import *
from lib.data_loader import DataLoader
from lib.small_predict import predict_small, print_predict_small
from lib.activation import ActivationOutput, RecurrentActivation
from lib.small_predict import build_and_predict, gen_multi_model, save_model, generate_metric
from lib.common_env_sets import lstm_units, dense_units, steps, metrics, dropout
from lib.common_env_sets import learning_rate, last_lstm_return_sequences
from lib.common_env_sets import loss, output_dense_activation, epochs, rand_seed


print(f"imported library. ({datetime.now()})")

2025-06-05 23:30:57.926663: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


imported library. (2025-06-05 23:31:00.620752)


In [2]:
##
## 모델링 환경 설정
##
window=1 # default = 1 , help = "time stamps"
data_dir='./csv/selectd5.csv'
    
mode='predict' # help = "back-test or predict")

mode2='sampling' # help = "greed or sampling")
verb='verbose' # help = "verbose or not_verb")
    
training_length=0.95 # default = 0.9)
from_pos = 0
last = [[8, 11, 14, 17, 36, 39],
        [1, 5, 18, 20, 30, 35],
        [7, 9, 24, 40, 42, 44],
        [3, 6, 7, 11, 12, 17],
        [3, 13, 28, 34, 38, 42],
        [5, 12, 24, 29, 32, 42]]
print(f"finished to set environemnt. ({datetime.now()})")


finished to set environemnt. (2025-06-05 23:31:10.796978)


In [3]:
dataset_dicts = {
    1: DataLoader(data_dir=data_dir,
                  training_length=training_length,
                  window_prev=window,
                  mode=mode,
                  from_pos=0,
                  verbose=False
                  ),
    2: DataLoader(data_dir=data_dir,
                  training_length=training_length,
                  window_prev=window,
                  mode=mode,
                  from_pos=300,
                  verbose=False
                  )
    }

print(f'completed to load data. {datetime.now()}')

completed to load data. 2025-06-05 23:31:14.486241


In [15]:
###########
## layers
## LSTM Neural 계층 선언
###########
datasets = [dataset_dicts[1], # 0
            dataset_dicts[2], # 1
           ]

lstm_args = {6: {"model":{"learning_rate": learning_rate[0],
                          "dropout": dropout[1],
                          "steps": steps[0], # last output elements count
                          "metrics": metrics[0],
                          "last_lstm_return_sequences": last_lstm_return_sequences[0],
                          "lstm_units": lstm_units[5],
                          "dense_units": dense_units[4],
                          "output_dense_activation": output_dense_activation[3],
                          "loss": loss[1],
                          "return_state": False,
                          "lstm_model": "Bidirectional",
                          "rand_seed": rand_seed[0] # rand_seed[0]
                          },
                 "train": {"epochs": epochs[3],
                           "is_shuffle": False,
                           "batch": 3,
                           "steps_per_epoch": 90 #100
                           },
                 "dataset": datasets[0]
                 }
             }

print(f'completed to set env for all models. {datetime.now()}')

completed to set env for all models. 2025-06-06 02:07:15.659868


In [16]:
def gen_metric(test_id, gen_num, round_limit, lstm_args=lstm_args):
    """ gen_metric """
    model_version_from = 1
    model_version_end = 1
    print(f'gen_metric.step.0 [{datetime.now()}]')
    args = {'test_id': test_id,
            'version': model_version_from,
            'write_to_db': True,
            'last': last,
            'mode2': mode2,
            'gen_num': gen_num,
            'round_limit': round_limit,
            'db_file_name': "./db/metrics.db"
            }
    print(f'gen_metric.step.1 [{datetime.now()}]')
    generate_metric(args=args,
                    from_version=model_version_from,
                    to_version=model_version_end,
                    lstm_args=lstm_args)
    print(f'gen_metric.step.2 [{datetime.now()}]')

In [ ]:
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
test_id='P4_5_8_1'
gen_num=5
round_limit=1000
gen_metric(test_id=test_id, gen_num=gen_num, round_limit=round_limit)

In [17]:
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
test_id='P4_5_8_2'
gen_num=1
round_limit=1000
gen_metric(test_id=test_id, gen_num=gen_num, round_limit=round_limit)

gen_metric.step.0 [2025-06-06 02:07:21.186954]
gen_metric.step.1 [2025-06-06 02:07:21.187356]
Test Start: 1~1
CPU 시간: 243.9375 초
메모리 사용량: 1163240 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-06 02:07:21.187460
********************
start to train all models. 2025-06-06 02:07:21.187470
6's training. status=start 2025-06-06 02:07:21.187478
6's training. status=end 2025-06-06 02:07:31.512736
completed to train all models. 2025-06-06 02:07:31.512825
train to create model(1). 2025-06-06 02:07:34.513120
INFO:tensorflow:Assets written to: ram://43b914d3749f410c9a7ea48c31f618eb/assets


INFO:tensorflow:Assets written to: ram://43b914d3749f410c9a7ea48c31f618eb/assets


CPU 시간: 278.515625 초
메모리 사용량: 1182696 KB
gen_metric.step.2 [2025-06-06 02:08:40.974856]


In [18]:
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
test_id='P4_5_8_3'
gen_num=1
round_limit=1000
gen_metric(test_id=test_id, gen_num=gen_num, round_limit=round_limit)

gen_metric.step.0 [2025-06-06 02:10:25.684145]
gen_metric.step.1 [2025-06-06 02:10:25.684304]
Test Start: 1~1
CPU 시간: 278.78125 초
메모리 사용량: 1182696 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-06 02:10:25.684374
********************
start to train all models. 2025-06-06 02:10:25.684383
6's training. status=start 2025-06-06 02:10:25.684390
6's training. status=end 2025-06-06 02:10:35.517757
completed to train all models. 2025-06-06 02:10:35.517847
train to create model(1). 2025-06-06 02:10:38.517905
INFO:tensorflow:Assets written to: ram://abd0eaf73214454691925600f99a9553/assets


INFO:tensorflow:Assets written to: ram://abd0eaf73214454691925600f99a9553/assets


CPU 시간: 313.421875 초
메모리 사용량: 1198276 KB
gen_metric.step.2 [2025-06-06 02:11:43.326464]


In [19]:
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
test_id='P4_5_8_4'
gen_num=1
round_limit=1000
gen_metric(test_id=test_id, gen_num=gen_num, round_limit=round_limit)

gen_metric.step.0 [2025-06-06 02:11:50.197122]
gen_metric.step.1 [2025-06-06 02:11:50.197373]
Test Start: 1~1
CPU 시간: 313.734375 초
메모리 사용량: 1198276 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-06 02:11:50.197439
********************
start to train all models. 2025-06-06 02:11:50.197447
6's training. status=start 2025-06-06 02:11:50.197454
6's training. status=end 2025-06-06 02:12:00.343797
completed to train all models. 2025-06-06 02:12:00.343877
train to create model(1). 2025-06-06 02:12:03.344381
INFO:tensorflow:Assets written to: ram://c2260105d66a4798a484b57f5b854e7f/assets


INFO:tensorflow:Assets written to: ram://c2260105d66a4798a484b57f5b854e7f/assets


CPU 시간: 348.65625 초
메모리 사용량: 1214388 KB
gen_metric.step.2 [2025-06-06 02:13:09.520634]


In [20]:
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
test_id='P4_5_8_5'
gen_num=1
round_limit=1000
gen_metric(test_id=test_id, gen_num=gen_num, round_limit=round_limit)

gen_metric.step.0 [2025-06-06 02:13:26.775072]
gen_metric.step.1 [2025-06-06 02:13:26.775542]
Test Start: 1~1
CPU 시간: 348.9375 초
메모리 사용량: 1214388 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-06 02:13:26.775613
********************
start to train all models. 2025-06-06 02:13:26.775621
6's training. status=start 2025-06-06 02:13:26.775629
6's training. status=end 2025-06-06 02:13:37.211419
completed to train all models. 2025-06-06 02:13:37.211499
train to create model(1). 2025-06-06 02:13:40.212066
INFO:tensorflow:Assets written to: ram://7c00140fe8f641efbfc7d2fc8fa2e15f/assets


INFO:tensorflow:Assets written to: ram://7c00140fe8f641efbfc7d2fc8fa2e15f/assets


CPU 시간: 383.203125 초
메모리 사용량: 1228328 KB
gen_metric.step.2 [2025-06-06 02:14:46.162217]


In [21]:
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
test_id='P4_5_8_6'
gen_num=1
round_limit=1000
gen_metric(test_id=test_id, gen_num=gen_num, round_limit=round_limit)

gen_metric.step.0 [2025-06-06 02:14:51.369412]
gen_metric.step.1 [2025-06-06 02:14:51.369721]
Test Start: 1~1
CPU 시간: 383.375 초
메모리 사용량: 1228328 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-06 02:14:51.369791
********************
start to train all models. 2025-06-06 02:14:51.369799
6's training. status=start 2025-06-06 02:14:51.369806
6's training. status=end 2025-06-06 02:15:01.692008
completed to train all models. 2025-06-06 02:15:01.692086
train to create model(1). 2025-06-06 02:15:04.692406
INFO:tensorflow:Assets written to: ram://d6058c02ed9e47e28fbb5ec172ece60f/assets


INFO:tensorflow:Assets written to: ram://d6058c02ed9e47e28fbb5ec172ece60f/assets


CPU 시간: 418.546875 초
메모리 사용량: 1247088 KB
gen_metric.step.2 [2025-06-06 02:16:11.276324]


In [22]:
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
test_id='P4_5_8_7'
gen_num=1
round_limit=1000
gen_metric(test_id=test_id, gen_num=gen_num, round_limit=round_limit)

gen_metric.step.0 [2025-06-06 02:16:30.666279]
gen_metric.step.1 [2025-06-06 02:16:30.666535]
Test Start: 1~1
CPU 시간: 418.734375 초
메모리 사용량: 1247088 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-06 02:16:30.666615
********************
start to train all models. 2025-06-06 02:16:30.666624
6's training. status=start 2025-06-06 02:16:30.666639
6's training. status=end 2025-06-06 02:16:41.910890
completed to train all models. 2025-06-06 02:16:41.910974
train to create model(1). 2025-06-06 02:16:44.911198
INFO:tensorflow:Assets written to: ram://f248e39eb0f74e30bd8ecceaaa5dc821/assets


INFO:tensorflow:Assets written to: ram://f248e39eb0f74e30bd8ecceaaa5dc821/assets


CPU 시간: 453.703125 초
메모리 사용량: 1263492 KB
gen_metric.step.2 [2025-06-06 02:17:52.385885]


In [23]:
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
test_id='P4_5_8_8'
gen_num=1
round_limit=1000
gen_metric(test_id=test_id, gen_num=gen_num, round_limit=round_limit)

gen_metric.step.0 [2025-06-06 02:17:59.652541]
gen_metric.step.1 [2025-06-06 02:17:59.652812]
Test Start: 1~1
CPU 시간: 454.0 초
메모리 사용량: 1263492 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-06 02:17:59.652880
********************
start to train all models. 2025-06-06 02:17:59.652888
6's training. status=start 2025-06-06 02:17:59.652896
6's training. status=end 2025-06-06 02:18:10.201721
completed to train all models. 2025-06-06 02:18:10.201803
train to create model(1). 2025-06-06 02:18:13.201913
INFO:tensorflow:Assets written to: ram://bf5ebb48fa064de691fc311bfed87bff/assets


INFO:tensorflow:Assets written to: ram://bf5ebb48fa064de691fc311bfed87bff/assets


CPU 시간: 489.15625 초
메모리 사용량: 1277732 KB
gen_metric.step.2 [2025-06-06 02:19:21.340037]


In [ ]:
## end